# Import Required Libraries
Import the necessary libraries, including pandas and sqlite3.

In [6]:
import sqlite3
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

db_files = [
    ("databases_uc/deliveroo.db", "deliveroo"),
    ("databases_uc/takeaway.db", "takeaway"),
    ("databases_uc/ubereats.db", "ubereats")
]
print(db_files)


for db_file, db_id in db_files:
    print(f"Inspecting schema for {db_file}...")
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"Tables: {tables}")
    
    if ('restaurants',) in tables:
        cursor.execute("SELECT sql FROM sqlite_master WHERE name='restaurants';")
        schema = cursor.fetchone()[0]
        print(f"Schema for 'restaurants':\n{schema}")
    else:
        print("No 'restaurants' table found.")
    
    conn.close()


[('databases_uc/deliveroo.db', 'deliveroo'), ('databases_uc/takeaway.db', 'takeaway'), ('databases_uc/ubereats.db', 'ubereats')]
Inspecting schema for databases_uc/deliveroo.db...
Tables: [('locations',), ('restaurants',), ('categories',), ('menu_items',), ('locations_to_restaurants',)]
Schema for 'restaurants':
CREATE TABLE restaurants(
                      id INTEGER PRIMARY KEY,
                      name TEXT,
                      visited_time,
                      latitude,
                      longitude,
                      menu_id,
                      category TEXT,
                      address TEXT,
                      postal_code,
                      prep_time,
                      delivery_time,
                      delivery_fee,
                      fulfillment_method,
                      min_order,
                      phonenumber,
                      rating,
                      rating_number,
                      uname
                      )
Inspec

# Define Database Connection
Establish a connection to the SQLite database using sqlite3.

In [7]:
import os
import sqlite3
import pandas as pd

def load_restaurant_data(db_file, db_id, c, m=50):
    """Load and calculate weighted ratings for restaurant data from specified database."""
    if not os.path.exists(db_file):
        print(f"Error: {db_file} does not exist.")
        return pd.DataFrame()

    queries = {
        "deliveroo": f"""
            SELECT 
                name AS restaurant_name, 
                address, postal_code, uname,
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                rating, CAST(rating_number AS INTEGER) AS rating_number,
                ((rating_number * rating) + ({m} * {c})) / (rating_number + {m}) AS weighted_rating,
                delivery_time, delivery_fee, min_order_value
            FROM restaurants
            WHERE rating IS NOT NULL AND rating_number > 0
        """,
        "takeaway": f"""
            SELECT 
                r.name AS restaurant_name, 
                r.address, r.city, r.primarySlug AS restaurant_id,
                CAST(r.longitude AS REAL) AS longitude, 
                CAST(r.latitude AS REAL) AS latitude, 
                r.ratings AS rating, CAST(r.ratingsNumber AS INTEGER) AS rating_number,
                ((r.ratingsNumber * r.ratings) + ({m} * {c})) / (r.ratingsNumber + {m}) AS weighted_rating,
                l.postalCode AS postal_code,
                r.delivery_time, r.delivery_fee, r.min_order_value
            FROM restaurants r
            LEFT JOIN (
                SELECT
                    lr.restaurant_id, l.postalCode
                FROM locations_to_restaurants lr
                JOIN locations l ON lr.location_id = l.ID
                WHERE l.postalCode IS NOT NULL
            ) l ON r.primarySlug = l.restaurant_id
            WHERE r.ratings IS NOT NULL AND r.ratingsNumber > 0
        """,
        "ubereats": f"""
            SELECT 
                title AS restaurant_name, 
                location__address AS address, location__city AS city,
                CAST(location__longitude AS REAL) AS longitude, 
                CAST(location__latitude AS REAL) AS latitude, 
                rating__rating_value AS rating, CAST(rating__review_count AS INTEGER) AS rating_number,
                ((rating__review_count * rating__rating_value) + ({m} * {c})) / (rating__review_count + {m}) AS weighted_rating,
                delivery_time, delivery_fee, min_order_value
            FROM restaurants
            WHERE rating__rating_value IS NOT NULL AND rating__review_count > 0
        """
    }

    try:
        conn = sqlite3.connect(db_file)
        df = pd.read_sql_query(queries[db_id], conn)
        if df.empty:
            print(f"Warning: No data found in {db_id}.")
        df['longitude'] = df['longitude'].round(6)
        df['latitude'] = df['latitude'].round(6)
        df['source'] = db_id
        return df
    except sqlite3.OperationalError as e:
        print(f"Error with {db_id}: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

# Define database files and identifiers
db_files = [
    ("databases_uc/deliveroo.db", "deliveroo"),
    ("databases_uc/takeaway.db", "takeaway"),
    ("databases_uc/ubereats.db", "ubereats")
]

# Load and combine data
dataframes = [load_restaurant_data(file, db_id, c=4.0, m=50) for file, db_id in db_files]
stacked_df = pd.concat(dataframes, ignore_index=True)

# Fill missing values
stacked_df['city'] = stacked_df.get('city', pd.Series()).fillna("Unknown")
stacked_df['postal_code'] = stacked_df.get('postal_code', pd.Series()).fillna("Unknown")
stacked_df['address'] = stacked_df['address'].fillna("Unknown")
stacked_df['rating'] = stacked_df['rating'].fillna(0.0)
stacked_df['delivery_time'] = stacked_df['delivery_time'].fillna("Unknown")
stacked_df['delivery_fee'] = stacked_df['delivery_fee'].fillna(0.0)
stacked_df['min_order_value'] = stacked_df['min_order_value'].fillna(0.0)

# Drop duplicates based on certain columns
stacked_df = stacked_df.drop_duplicates(subset=['restaurant_name', 'address', 'rating_number', 'source'])

# Sort the DataFrame by weighted_rating in descending order
stacked_df = stacked_df.sort_values(by='weighted_rating', ascending=False)

# Display the updated DataFrame
print(f"Total rows after processing: {stacked_df.shape[0]}")
print(stacked_df.head())


DatabaseError: Execution failed on sql '
            SELECT 
                name AS restaurant_name, 
                address, postal_code, uname,
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                rating, CAST(rating_number AS INTEGER) AS rating_number,
                ((rating_number * rating) + (50 * 4.0)) / (rating_number + 50) AS weighted_rating,
                delivery_time, delivery_fee, min_order_value
            FROM restaurants
            WHERE rating IS NOT NULL AND rating_number > 0
        ': no such column: min_order_value

In [ ]:
def load_restaurant_data(db_file, db_id, c, m=50):
    """Load and calculate weighted ratings for restaurant data from specified database."""
    if not os.path.exists(db_file):
        print(f"Error: {db_file} does not exist.")
        return pd.DataFrame()

    queries = {
        "deliveroo": f"""
            SELECT 
                name AS restaurant_name, 
                address, postal_code, uname,
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                rating, CAST(rating_number AS INTEGER) AS rating_number,
                ((rating_number * rating) + ({m} * {c})) / (rating_number + {m}) AS weighted_rating,
                prep_time, delivery_time, min_order
            FROM restaurants
            WHERE rating IS NOT NULL AND rating_number > 0
        """,
        "takeaway": f"""
            SELECT 
                r.name AS restaurant_name, 
                r.address, r.city, r.primarySlug AS restaurant_id,
                CAST(r.longitude AS REAL) AS longitude, 
                CAST(r.latitude AS REAL) AS latitude, 
                r.ratings AS rating, CAST(r.ratingsNumber AS INTEGER) AS rating_number,
                ((r.ratingsNumber * r.ratings) + ({m} * {c})) / (r.ratingsNumber + {m}) AS weighted_rating,
                l.postalCode AS postal_code
            FROM restaurants r
            LEFT JOIN (
                SELECT
                    lr.restaurant_id, l.postalCode
                FROM locations_to_restaurants lr
                JOIN locations l ON lr.location_id = l.ID
                WHERE l.postalCode IS NOT NULL
            ) l ON r.primarySlug = l.restaurant_id
            WHERE r.ratings IS NOT NULL AND r.ratingsNumber > 0
        """,
        "ubereats": f"""
            SELECT 
                title AS restaurant_name, 
                location__address AS address, location__city AS city,
                CAST(location__longitude AS REAL) AS longitude, 
                CAST(location__latitude AS REAL) AS latitude, 
                rating__rating_value AS rating, CAST(rating__review_count AS INTEGER) AS rating_number,
                ((rating__review_count * rating__rating_value) + ({m} * {c})) / (rating__review_count + {m}) AS weighted_rating
            FROM restaurants
            WHERE rating__rating_value IS NOT NULL AND rating__review_count > 0
        """
    }

    try:
        conn = sqlite3.connect(db_file)
        df = pd.read_sql_query(queries[db_id], conn)
        if df.empty:
            print(f"Warning: No data from {db_id}.")
        df['longitude'] = df['longitude'].round(6)
        df['latitude'] = df['latitude'].round(6)
        df['source'] = db_id
        return df
    except sqlite3.OperationalError as e:
        print(f"Error with {db_id}: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

# Define database files and identifiers
db_files = [
    ("databases_uc/deliveroo.db", "deliveroo"),
    ("databases_uc/takeaway.db", "takeaway"),
    ("databases_uc/ubereats.db", "ubereats")
]

# Load and combine data
dataframes = [load_restaurant_data(file, db_id, c=4.0, m=50) for file, db_id in db_files]
stacked_df = pd.concat(dataframes, ignore_index=True)

# Fill missing values
stacked_df['city'] = stacked_df.get('city', pd.Series()).fillna("Unknown")
stacked_df['postal_code'] = stacked_df.get('postal_code', pd.Series()).fillna("Unknown")
stacked_df['address'] = stacked_df['address'].fillna("Unknown")
stacked_df['rating'] = stacked_df['rating'].fillna(0.0)

# Drop duplicates based on restaurant_name, address, rating and source
stacked_df = stacked_df.drop_duplicates(subset=['restaurant_name', 'address', 'rating_number', 'source'])

# Sort the DataFrame by weighted_rating in descending order
stacked_df = stacked_df.sort_values(by='weighted_rating', ascending=False)

# Display the updated DataFrame
print(f"Total rows after dropping duplicates: {stacked_df.shape[0]}")
stacked_df = stacked_df.sort_values(by='weighted_rating', ascending=False)

# Display the sorted DataFrame
stacked_df.head()


Total rows after dropping duplicates: 6413


,restaurant_name,address,postal_code,uname,longitude,latitude,rating,rating_number,weighted_rating,prep_time,delivery_time,min_order,source,city,restaurant_id
59738,De Echte Eethuis Carlos,Herseltsesteenweg 207,2230,NaN,4.835920,51.002308,5.0,599,4.922958,NaN,NaN,NaN,takeaway,Aarschot,eethuis-carlos-aarschot
8438,Veziroglu,Franklin Rooseveltlaan 100,1800,NaN,4.425172,50.931044,5.0,455,4.900990,NaN,NaN,NaN,takeaway,Vilvorde,veziroglu
2633,Snack Thieffry,Rue Pervyse 62,1140,NaN,4.399869,50.830792,4.9,3875,4.888535,NaN,NaN,NaN,takeaway,1040,snack-thieffry
4068,Sushiline,Chaussée De Louvain 470,1140,NaN,4.391012,50.853172,4.9,3036,4.885418,NaN,NaN,NaN,takeaway,Bruxelles,sushiline-bruxelles
60904,Snack Dekan,Place Cardinal Mercier 41,1731,NaN,4.329693,50.880092,4.9,2845,4.884456,NaN,NaN,NaN,takeaway,Jette,snack-dekan


In [ ]:
# Count the number of 'Unknown' values in each column
unknown_counts = stacked_df.apply(lambda x: x[x == 'Unknown'].count())
print("Count of 'Unknown' values in each column:")
print(unknown_counts)

In [ ]:
stacked_df.info()

In [ ]:
stacked_df.describe()

# Top Pizza Restaurants Per Delivery App 



In [ ]:
# Filter for pizza restaurants
pizza_restaurants = stacked_df[stacked_df['restaurant_name'].str.contains('pizza', case=False, na=False)]

# Get top 5 pizza restaurants per delivery app based on weighted rating
top_pizza_per_app = pizza_restaurants.groupby('source').apply(lambda x: x.nlargest(5, 'weighted_rating')).reset_index(drop=True)

# Display the top pizza restaurants per delivery app
print("Top Pizza Restaurants Per Delivery App:")
print(top_pizza_per_app[['restaurant_name', 'address', 'city', 'rating', 'rating_number', 'weighted_rating', 'source']])

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Set the aesthetic style of the plots
sns.set(style="whitegrid")

# Create a bar plot for the top 5 pizza restaurants per delivery app
plt.figure(figsize=(14, 8))
sns.barplot(x='weighted_rating', y='restaurant_name', hue='source', data=top_pizza_per_app, dodge=False)

# Add titles and labels
plt.title('Top 5 Pizza Restaurants Per Delivery App', fontsize=16)
plt.xlabel('Weighted Rating', fontsize=14)
plt.ylabel('Restaurant Name', fontsize=14)

# Display the legend
plt.legend(title='Delivery App', loc='lower right')

# Show the plot
plt.show()

# Rating Distributions per App 

In [ ]:
import sqlite3
import pandas as pd

# Step 1: Load Databases
deliveroo_db = 'databases_uc/deliveroo.db'
takeaway_db = 'databases_uc/takeaway.db'
ubereats_db = 'databases_uc/ubereats.db'

# Step 2: Function to Fetch Restaurant Ratings from Each Database
def fetch_restaurant_ratings(db_file, db_id):
    conn = sqlite3.connect(db_file)
    query = None
    if db_id == "deliveroo":
        query = """
        SELECT name AS restaurant_name, rating, CAST(rating_number AS INTEGER) AS rating_count
        FROM restaurants
        WHERE rating IS NOT NULL AND rating_number > 0;
        """
    elif db_id == "takeaway":
        query = """
        SELECT name AS restaurant_name, ratings AS rating, CAST(ratingsNumber AS INTEGER) AS rating_count
        FROM restaurants
        WHERE ratings IS NOT NULL AND ratingsNumber > 0;
        """
    elif db_id == "ubereats":
        query = """
        SELECT title AS restaurant_name, rating__rating_value AS rating, CAST(rating__review_count AS INTEGER) AS rating_count
        FROM restaurants
        WHERE rating__rating_value IS NOT NULL AND rating__review_count > 0;
        """
    else:
        return pd.DataFrame()  # Return empty DataFrame if db_id is invalid
    df = pd.read_sql_query(query, conn)
    conn.close()
    df['source'] = db_id
    return df

# Step 3: Fetch Data from All Platforms
df_deliveroo = fetch_restaurant_ratings(deliveroo_db, "deliveroo")
df_takeaway = fetch_restaurant_ratings(takeaway_db, "takeaway")
df_ubereats = fetch_restaurant_ratings(ubereats_db, "ubereats")

# Combine data into one DataFrame
df_ratings = pd.concat([df_deliveroo, df_takeaway, df_ubereats], ignore_index=True)

# Step 4: Normalize Restaurant Names for Cross-Platform Comparison
df_ratings['restaurant_name_clean'] = df_ratings['restaurant_name'].str.lower().str.strip()

# Analysis 1: Rating Distribution Across Platforms
rating_distribution = df_ratings.groupby('source')['rating'].describe()
print("Rating Distribution Across Platforms:")
print(rating_distribution)





In [ ]:
#Cross-Platform Restaurants (gebaseerd op cleaning v/d naam, strip en lower)

# Filter restaurants that appear on multiple platforms
cross_platform = df_ratings.groupby('restaurant_name_clean').filter(lambda x: x['source'].nunique() > 1)

# Create a pivot table for comparing ratings
cross_platform_pivot = cross_platform.pivot_table(
    index='restaurant_name_clean', 
    columns='source', 
    values='rating', 
    aggfunc='mean'
).dropna()

print("\nCross-Platform Rating Comparison:")
print(cross_platform_pivot)


# Create a pivot table for comparing rating counts
cross_platform_count_pivot = cross_platform.pivot_table(
    index='restaurant_name_clean', 
    columns='source', 
    values='rating_count', 
    aggfunc='mean'
).dropna()

print("\nCross-Platform Rating Count Comparison:")
print(cross_platform_count_pivot)

    Rating Differences Across Platforms:
        UberEats has the highest average rating (4.30), indicating greater customer satisfaction.
        Deliveroo has a moderate average rating (4.18), showing consistency.
        Takeaway has the lowest average rating (4.08) with higher variability, indicating mixed customer experiences.

    Review Count Variations:
        Takeaway generally has the most reviews, indicating a larger or more engaged customer base compared to Deliveroo and UberEats.
        UberEats tends to have fewer reviews but higher ratings, suggesting selective or more positive feedback.

    Top Restaurants:
        The top-rated restaurants are mostly from Takeaway, especially those with high review counts.
        Deliveroo and UberEats have top-rated restaurants, but with fewer reviews, indicating potential for growth in popularity.

    Cross-Platform Variability:
        Some restaurants have significantly different ratings across platforms, indicating differences in customer demographics or service quality.
 > De
 

In [ ]:
# Analysis 2: Cross-Platform Comparison for Similar Restaurants
# Filter restaurants that appear on multiple platforms
cross_platform = df_ratings.groupby('restaurant_name_clean').filter(lambda x: x['source'].nunique() > 1)

# Create a pivot table for comparing ratings and review counts
cross_platform_pivot = cross_platform.pivot_table(
    index='restaurant_name_clean', 
    columns='source', 
    values=['rating', 'rating_count'], 
    aggfunc='mean'
).dropna()

print("\nCross-Platform Rating and Review Count Comparison:")
print(cross_platform_pivot)

# Combine cross-platform ratings and review counts into a single DataFrame
cross_platform_combined = cross_platform_pivot.reset_index()
cross_platform_combined.columns = ['restaurant_name_clean', 'rating_deliveroo', 'rating_takeaway', 'rating_ubereats', 'rating_count_deliveroo', 'rating_count_takeaway', 'rating_count_ubereats']

# Display the combined DataFrame
print("\nCross-Platform Ratings and Review Counts:")
print(cross_platform_combined)


In [ ]:
def plot_metrics(statistics, metrics, platforms):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Descriptive Statistics for Delivery Platforms', fontsize=16)

    for i, metric in enumerate(metrics):
        ax = axes[i // 2, i % 2]
        has_data = False  # Track if any data is available for the metric
        for platform in platforms:
            stats = statistics[platform].get(metric.lower().replace(" ", "_"), {})
            # Check if stats is a non-empty dictionary
            if isinstance(stats, dict) and stats:  
                has_data = True
                ax.bar(platform, stats.get('mean', 0), yerr=stats.get('std', 0), capsize=5, label=platform)
        if has_data:
            ax.set_title(metric)
            ax.set_ylabel('Value')
            ax.legend()
        else:
            ax.set_title(f"{metric} (No Data Available)")
            ax.set_ylabel('Value')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # Define the metrics and platforms
    metrics = ['Delivery Fee', 'Delivery Time', 'Minimum Order Value', 'Ratings']
    platforms = ['Deliveroo', 'Takeaway', 'UberEats']

    # Plot the metrics
    plot_metrics(statistics, metrics, platforms)